In [5]:
import glob
import os
import shutil

In [10]:
base_path = "/run/user/715136/gvfs/smb-share:server=wfs-biologie.top.gwdg.de,share=ubps-all$/Language_Acquisition/HIWI/HIWIstudies/SmallFaces in the Wild HIWI/Data/"

neon_paths = glob.glob(os.path.join(base_path,"Neon/*/*Scene*.mp4")) #neon
names_neon = ["Neon_" + path.split("/")[-2].replace("-","_") for path in neon_paths]

other_cams_paths = glob.glob(os.path.join(base_path,"Pupil Core/raspi-*/*/*/world.mp4"))
names_other = []
for other_cams_path in other_cams_paths:
    splitted_name = other_cams_path.split("/")
    name = f'{splitted_name[-4].replace("-","_")}_{splitted_name[-3]}_{splitted_name[-2]}'
    names_other.append(name)

names = names_neon + names_other
pahts_all = neon_paths + other_cams_paths

In [4]:
for name, path in zip(names, pahts_all):
    shutil.copy(path, f"/usr/users/vhassle/datasets/Wortschatzinsel/all_videos/{name}.mp4")


NameError: name 'paths_all' is not defined